In [ ]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup as soup
import time
from selenium.webdriver.common.by import By

In [ ]:
def nse_stock_price(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    prefs = {"download.default_directory" : os.getcwd()}
    chrome_options.add_experimental_option("prefs",prefs)
    browser = webdriver.Chrome('C:/Users/prabhat/Downloads/chromedriver_win32/chromedriver.exe', options=chrome_options)
    browser.get(url)
    # waiting for page load
    time.sleep(10)
    ###### NOT WORKING WITHOUT SCROLL
    browser.execute_script("window.scrollBy(0,800)","")
    # getting historical data
    historical_data_page = browser.find_elements(By.XPATH,'//*[@id="subtab-equity"]/div[2]/nav/div/div/a[5]')
    historical_data_page[0].click()
    time.sleep(2)
    # setting range to 1 week
    select_range = browser.find_element(By.XPATH,'//*[@id="historical-trade"]/section/div/div[1]/div/div[1]/ul/li[2]/a')
    select_range.click()
    # applying filter
    filter_button = browser.find_element(By.XPATH,'//*[@id="equity-historical-Date-filter"]/div[3]/button')
    filter_button.click()
    time.sleep(10)
    browser.execute_script("window.scrollBy(0,400)","")
    time.sleep(10)
    # table content from historical data
    # table data
    table_stock_data = browser.find_element(By.XPATH,'//*[@id="equityHistoricalTable"]/tbody')
    table_stock_data = soup(table_stock_data.get_attribute('innerHTML'),'html.parser')
    stock_data = []
    for column_data in table_stock_data.findAll('td'):
        stock_data.append(column_data.text)
    browser.quit()
    # return column names from table and content of table
    return stock_data

In [ ]:
stock_data1 = nse_stock_price('https://www.nseindia.com/get-quotes/equity?symbol=POWERGRID')
stock_data2 = nse_stock_price('https://www.nseindia.com/get-quotes/equity?symbol=SBIN')

In [ ]:
stock_data1

In [ ]:
stock_data2

In [ ]:
import random


class Node(object):
    '''
    Class to implement node
    '''

    def __init__(self, key, level):
        self.key = key

        # list to hold references to node of different level
        self.forward = [None] * (level + 1)


class SkipList(object):
    '''
    Class for Skip list
    '''

    def __init__(self, max_lvl, P):
        # Maximum level for this skip list
        self.MAXLVL = max_lvl

        # P is the fraction of the nodes with level
        # i references also having level i+1 references
        self.P = P

        # create header node and initialize key to -1
        self.header = self.createNode(self.MAXLVL, -1)

        # current level of skip list
        self.level = 0

    # create  new node
    def createNode(self, lvl, key):
        n = Node(key, lvl)
        return n

    # create random level for node
    def randomLevel(self):
        lvl = 0
        while random.random() < self.P and \
                lvl < self.MAXLVL: lvl += 1
        return lvl

    # insert given key in skip list
    def insertElement(self, key):
        # create update array and initialize it
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        start from highest level of skip list
        move the current reference forward while key 
        is greater than key of node next to current
        Otherwise inserted current in update and 
        move one level down and continue search
        '''
        for i in range(self.level, -1, -1):
            while current.forward[i] and \
                    current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        ''' 
        reached level 0 and forward reference to 
        right, which is desired position to 
        insert key.
        '''
        current = current.forward[0]

        '''
        if current is NULL that means we have reached
           to end of the level or current's key is not equal
           to key to insert that means we have to insert
           node between update[0] and current node
       '''
        if current == None or current.key != key:
            # Generate a random level for node
            rlevel = self.randomLevel()

            '''
            If random level is greater than list's current
            level (node with highest level inserted in 
            list so far), initialize update value with reference
            to header for further use
            '''
            if rlevel > self.level:
                for i in range(self.level + 1, rlevel + 1):
                    update[i] = self.header
                self.level = rlevel

            # create new node with random level generated
            n = self.createNode(rlevel, key)

            # insert node by rearranging references
            for i in range(rlevel + 1):
                n.forward[i] = update[i].forward[i]
                update[i].forward[i] = n

            print("Successfully inserted key {}".format(key))

    def deleteElement(self, search_key):

        # create update array and initialize it
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        start from highest level of skip list
        move the current reference forward while key 
        is greater than key of node next to current
        Otherwise inserted current in update and 
        move one level down and continue search
        '''
        for i in range(self.level, -1, -1):
            while (current.forward[i] and current.forward[i].key < search_key):
                current = current.forward[i]
            update[i] = current

        ''' 
        reached level 0 and advance reference to 
        right, which is possibly our desired node
        '''
        current = current.forward[0]

        # If current node is target node
        if current != None and current.key == search_key:

            '''
            start from lowest level and rearrange references 
            just like we do in singly linked list
            to remove target node
            '''
            for i in range(self.level + 1):

                '''
                If at level i, next node is not target 
                node, break the loop, no need to move 
                further level
                '''
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]

            # Remove levels having no elements
            while (self.level > 0 and self.header.forward[self.level] == None):
                self.level -= 1
            print("Successfully deleted {}".format(search_key))

    def searchElement(self, key):
        current = self.header

        for i in range(self.level, -1, -1):
            while (current.forward[i] and current.forward[i].key < key):
                current = current.forward[i]

        # reached level 0 and advance reference to
        # right, which is prssibly our desired node
        current = current.forward[0]

        # If current node have key equal to
        # search key, we have found our target node
        if current and current.key == key:
            print("Found key ", key)
        else:
            print('Not Found')

    # Display skip list level wise
    def displayList(self):
        print("\n*****Skip List******")
        head = self.header
        for lvl in range(self.level + 1):
            print("Level {}: ".format(lvl), end=" ")
            node = head.forward[lvl]
            while (node != None):
                print(node.key, end=" ")
                node = node.forward[lvl]
            print("")

    

In [ ]:
stock1=SkipList(3,0.5)
for i in range(5,len(stock_data1),14):
    stock1.insertElement(float(stock_data1[i]))

In [ ]:
stock1.displayList()

In [ ]:
stock2=SkipList(3,0.5)
for i in range(5,len(stock_data2),14):
    stock2.insertElement(float(stock_data2[i]))

In [ ]:
stock2.displayList()

In [ ]:
stock2.searchElement(558)